In [ ]:
# With ipython notebooks
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=""

# With python
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ['CUDA_VISIBLE_DEVICES'] = "0,3"  # specify which GPU(s) to be used

# With bash
# export CUDA_DEVICE_ORDER="PCI_BUS_ID"
# export CUDA_VISIBLE_DEVICES="0,3" 

import os
import numpy as np
import tensorflow as tf
# tf.enable_eager_execution()
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior() 
# tf.enable_eager_execution()


from albert_act_v2 import (
    modeling, 
    optimization, 
    tokenization
)

# IPython automatically reload all changed code
%load_ext autoreload
%autoreload 2

In [ ]:
tf.__version__

In [ ]:
albert_config = modeling.AlbertConfig.from_json_file('./albert_config.json')

In [ ]:
tf.compat.v1.disable_eager_execution()

input_ids = tf.compat.v1.placeholder(tf.int32, [None, None], "input_ids")
input_mask = tf.compat.v1.placeholder(tf.int32, [None, None], "input_mask")
segment_ids = tf.compat.v1.placeholder(tf.int32, [None, None], "segment_ids")
mlm_positions = tf.compat.v1.placeholder(tf.int32, [None, None], "mlm_positions")

model = modeling.AlbertModelWithACT(
    config=albert_config,
    is_training=False,
    input_ids=input_ids,
    input_mask=input_mask,
    token_type_ids=segment_ids,
    use_one_hot_embeddings=False)

In [ ]:
tvars = tf.compat.v1.trainable_variables()
tvars

In [ ]:
INIT_CHECKPOINT = "./model.ckpt-112500"

In [ ]:
(assignment_map, initialized_variable_names
) = modeling.get_assignment_map_from_checkpoint(tvars, INIT_CHECKPOINT)

In [ ]:
assignment_map

In [ ]:
reader = tf.train.load_checkpoint("./model.ckpt-112500")
shapes = reader.get_variable_to_shape_map()
dtypes = reader.get_variable_to_dtype_map()

In [ ]:
reader.get_tensor('bert/embeddings/LayerNorm/gamma')

In [ ]:
for (k, v) in assignment_map.items():
  file_name = os.path.join('./weights/', '{}.npy'.format(v.strip().replace('/', '-')))
  print(file_name)
  with open(file_name, 'wb') as f:
    np.save(f, reader.get_tensor(k))

In [ ]:
with open(os.path.join('./weights/', '{}.npy'.format('transformer-layer_normalization-beta')), 'rb') as f:
    save_param = np.load(f)
print(save_param[:10])